# Final Project
Abigail Allen  
Josh Urry  
Trevor Jex

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Load data
df = pd.read_csv("Motor_Vehicle_Crashes_-_Case_Information__Three_Year_Window.csv")

In [3]:
df.head()

,Year,Crash Descriptor,Time,Date,Day of Week,Police Report,Lighting Conditions,Municipality,Collision Type Descriptor,County Name,Road Descriptor,Weather Conditions,Traffic Control Device,Road Surface Conditions,DOT Reference Marker Location,Pedestrian Bicyclist Action,Event Descriptor,Number of Vehicles Involved
0,2017,Property Damage & Injury Accident,6:49,01/03/2017,Tuesday,Y,Dawn,HEMPSTEAD,REAR END,NASSAU,Straight and Level,Cloudy,Traffic Signal,Wet,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
1,2017,Injury Accident,17:17,01/02/2017,Monday,Y,Dark-Road Lighted,HEMPSTEAD,OTHER,NASSAU,Straight and Level,Rain,Stop Sign,Wet,NaN,"Crossing, With Signal","Pedestrian, Collision With",1
2,2017,Property Damage Accident,22:50,01/02/2017,Monday,Y,Dark-Road Lighted,HEMPSTEAD,LEFT TURN (3),NASSAU,Straight and Level,Rain,Traffic Signal,Wet,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
3,2017,Property Damage Accident,10:00,01/03/2017,Tuesday,Y,Daylight,HEMPSTEAD,OVERTAKING,NASSAU,Straight and Level,Rain,Traffic Signal,Wet,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
4,2017,Property Damage Accident,7:05,01/03/2017,Tuesday,Y,Daylight,HEMPSTEAD,REAR END,NASSAU,Straight and Level,Cloudy,None,Wet,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2


In [4]:
# Check number of samples
len(df)

1890537

## Preprocessing

In [5]:
# Check NA values
df.isna().sum()

Year                                   0
Crash Descriptor                       0
Time                                   0
Date                                   0
Day of Week                            0
Police Report                          0
Lighting Conditions                    0
Municipality                       14620
Collision Type Descriptor              0
County Name                            0
Road Descriptor                        0
Weather Conditions                     0
Traffic Control Device                 0
Road Surface Conditions                0
DOT Reference Marker Location    1257386
Pedestrian Bicyclist Action            0
Event Descriptor                       0
Number of Vehicles Involved            0
dtype: int64

Let's drop the "DOT Reference Marker Location" variable, since the majority of accidents have this missing. Maybe we could just remove the rows where the Municipality is blank. Or just drop it as well and just use the County.

In [6]:
df.drop("DOT Reference Marker Location", axis = 1, inplace = True)

Let's figure out how to handle the date variables.

In [7]:
df.Year.unique()

array([2017, 2019, 2021, 2018, 2020], dtype=int64)

We could represent these as 0, 1, 2, 3, and 4 as a variable that represents "years from 2017", so it is numeric.

In [8]:
df.Year.replace({2017: 0, 2018: 1, 2019: 2, 2020: 3, 2021: 4}, inplace = True)

Let's convert time and date to date time objects and extract the minutes, hours, month, and day

In [9]:
df.Time = pd.to_datetime(df.Time, infer_datetime_format = True)
df.Date = pd.to_datetime(df.Date, infer_datetime_format = True)

In [10]:
df["month"] = df.Date.dt.month
df["day"] = df.Date.dt.day
df["hour"] = df.Time.dt.hour
df["minute"] = df.Time.dt.minute # My guess is that we'll end up dropping this

In [11]:
# Drop Date and Time variables 
df.drop(["Date", "Time"], axis = 1, inplace = True)

In [12]:
# Now let's convert day of the week to numeric
df["Day of Week"].replace({"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4,
                          "Friday": 5, "Saturday": 6}, inplace = True)

In [13]:
df.head()

,Year,Crash Descriptor,Day of Week,Police Report,Lighting Conditions,Municipality,Collision Type Descriptor,County Name,Road Descriptor,Weather Conditions,Traffic Control Device,Road Surface Conditions,Pedestrian Bicyclist Action,Event Descriptor,Number of Vehicles Involved,month,day,hour,minute
0,0,Property Damage & Injury Accident,2,Y,Dawn,HEMPSTEAD,REAR END,NASSAU,Straight and Level,Cloudy,Traffic Signal,Wet,Not Applicable,"Other Motor Vehicle, Collision With",2,1,3,6,49
1,0,Injury Accident,1,Y,Dark-Road Lighted,HEMPSTEAD,OTHER,NASSAU,Straight and Level,Rain,Stop Sign,Wet,"Crossing, With Signal","Pedestrian, Collision With",1,1,2,17,17
2,0,Property Damage Accident,1,Y,Dark-Road Lighted,HEMPSTEAD,LEFT TURN (3),NASSAU,Straight and Level,Rain,Traffic Signal,Wet,Not Applicable,"Other Motor Vehicle, Collision With",2,1,2,22,50
3,0,Property Damage Accident,2,Y,Daylight,HEMPSTEAD,OVERTAKING,NASSAU,Straight and Level,Rain,Traffic Signal,Wet,Not Applicable,"Other Motor Vehicle, Collision With",2,1,3,10,0
4,0,Property Damage Accident,2,Y,Daylight,HEMPSTEAD,REAR END,NASSAU,Straight and Level,Cloudy,None,Wet,Not Applicable,"Other Motor Vehicle, Collision With",2,1,3,7,5


Let's figure out the target variable

In [14]:
df["Event Descriptor"].unique()

array(['Other Motor Vehicle, Collision With',
       'Pedestrian, Collision With',
       'Earth Embankment/Rock Cut/Ditch, Collision With Fixed Object',
       'Bicyclist, Collision With',
       'Fire Hydrant, Collision With Fixed Object',
       'Other Fixed Object*, Collision With Fixed Object',
       'Barrier, Collision With Fixed Object',
       'Curbing, Collision With Fixed Object',
       'Building/Wall, Collision With Fixed Object',
       'Light Support/Utility Pole, Collision With Fixed Object',
       'Median - Not At End, Collision With Fixed Object',
       'Tree, Collision With Fixed Object', 'Deer',
       'Overturned, Non-Collision',
       'Guide Rail - Not At End, Collision With Fixed Object',
       'Guide Rail - End, Collision With Fixed Object',
       'Animal, Collision With',
       'Bridge Structure, Collision With Fixed Object',
       'Other Object (Not Fixed)*, Collision With',
       'Snow Embankment, Collision With Fixed Object',
       'Sign Post, Colli

In [15]:
df["Crash Descriptor"].unique()

array(['Property Damage & Injury Accident', 'Injury Accident',
       'Property Damage Accident', 'Fatal Accident'], dtype=object)

Maybe we just do classes from the descriptor variable? That would definitely be a lot easier. It seems like it would be pretty messy to deal with the event descriptor variable.